In [ ]:
import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line


In [11]:
"""import movie file with budget info"""
df_budget = pd.read_csv('Data/movie_budgets.csv') # get the data for budget of movies
df_scraped = pd.read_csv('Data/All_movies_year_1990-2020.csv')
df_budget.rename(columns = {'movie': 'Title'}, inplace=True) # rename the movie column to title so it can be used as an index for joining 
df_budget.set_index('Title',inplace=True) # set the name as index

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
df_budget.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5782 entries, Avatar to My Date With Drew
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   production_budget  5782 non-null   object
 3   domestic_gross     5782 non-null   object
 4   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(4)
memory usage: 271.0+ KB


In [13]:
"""Clean budget data"""
df_budget['production_budget'] = df_budget['production_budget'].str.strip('$').str.replace(",","").astype(float)/1000000

In [10]:
# """Clean budget data"""
# df_budget['production_budget'] = df_budget['production_budget'].apply(lambda x: x.str.strip('$').replace(",","")) #Clean currency string   
# df_budget['production_budget'] = df_budget['production_budget'].astype(float)/1000000 # Convert to integer and change the amount into millions

In [14]:
type(df_budget['production_budget'][0])# check to see if amount is numeric

numpy.float64

In [15]:
"""Merge the df with budget info with the web scraped dataframe"""
fulldf = pd.merge(df_scraped,df_budget['production_budget'],on ='Title') # add the budget column and release date to df_scraped
fulldf.rename(columns = {'production_budget':'Budget in millions (USD)', 'Gross Income' : 'Gross Income in millions (USD)'}, inplace = True) # rename production budget to Budget

<IPython.core.display.Javascript object>

In [16]:
"""Add Profit percent column"""
fulldf['Profit percent'] = ((fulldf['Gross Income in millions (USD)'] - fulldf['Budget in millions (USD)'] )/fulldf['Budget in millions (USD)'])*100

In [17]:
fulldf.drop('Unnamed: 0', axis = 1,inplace = True)

In [18]:
"""Remove movies that have no runtime"""
fulldf = fulldf [fulldf['Movie Runtime'] > 0]

In [19]:
"""Replace unreported Gross income by the median Gross income""" 
fulldf=fulldf.replace({'Gross Income in millions (USD)': {0: fulldf['Gross Income in millions (USD)'].median()}})

In [20]:
"""Replace unreported Votes by the median Votes""" 
fulldf=fulldf.replace({'Votes': {0: fulldf['Votes'].median()}})

In [21]:
fulldf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4388 entries, 0 to 4397
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Title                           4388 non-null   object 
 1   Movie Rating                    4388 non-null   object 
 2   Year                            4388 non-null   object 
 3   Gross Income in millions (USD)  4388 non-null   float64
 4   Movie Genre                     4388 non-null   object 
 5   Movie Runtime                   4388 non-null   int64  
 6   IMDb Rating                     4388 non-null   float64
 7   Votes                           4388 non-null   float64
 8   Budget in millions (USD)        4388 non-null   float64
 9   Profit percent                  4388 non-null   float64
dtypes: float64(5), int64(1), object(4)
memory usage: 377.1+ KB


In [22]:
"""Save file to csv"""
#fulldf.to_csv("fulldf.csv",index=False) # save this df as an excel file
fulldf.to_csv("Data/newfulldf.csv",index=False) # save this df as an excel file